# ProxyWhirl Interface Notebook

A focused, copy/paste friendly guide to **every primary ProxyWhirl interface**.

This notebook covers:

- Python SDK (sync): `ProxyWhirl`
- Python SDK (async): `AsyncProxyWhirl`
- Strategy selection and hot-swap
- Pool maintenance and stats
- Fetch/validate pipeline
- CLI commands
- REST API usage (FastAPI)
- MCP wrapper usage

All network-sensitive examples are mocked so execution stays deterministic.


## How To Use This Notebook

1. Run top-to-bottom once to verify your environment.
2. Copy the sections you need into your own project.
3. Replace mocked request handlers with real endpoints when ready.


In [ ]:
# Optional Colab setup (kept commented for simplicity):
# !pip -q install -U proxywhirl[api] httpx fastapi
# !pip -q install -U proxywhirl
# Then restart runtime and run all cells.


In [ ]:
from __future__ import annotations

import asyncio
import json
import platform
import subprocess
import sys
import threading
from textwrap import shorten
from unittest.mock import patch

import httpx
from loguru import logger

import proxywhirl
from proxywhirl import (
    AsyncProxyWhirl,
    HealthStatus,
    Proxy,
    ProxySource,
    ProxyWhirl,
    RetryPolicy,
)

# Reduce noisy logs so notebook output is easier to read.
logger.remove()
logger.add(sys.stderr, level="WARNING")

print(f"Python: {platform.python_version()} ({platform.system()})")
print(f"ProxyWhirl: {proxywhirl.__version__}")

## Utilities

In [ ]:
def run_async(coro):
    """Run async code from scripts, notebooks, or already-running loops."""
    try:
        asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)

    result: dict[str, object] = {}
    error: dict[str, BaseException] = {}

    def _runner() -> None:
        try:
            result["value"] = asyncio.run(coro)
        except BaseException as exc:  # pragma: no cover - helper fallback
            error["value"] = exc

    thread = threading.Thread(target=_runner, daemon=True)
    thread.start()
    thread.join()

    if "value" in error:
        raise error["value"]
    return result.get("value")


def jprint(value) -> None:
    print(json.dumps(value, indent=2, default=str))


def run_cli(args: list[str], max_lines: int = 14) -> int:
    cmd = [sys.executable, "-m", "proxywhirl.cli"] + args
    print(f"$ proxywhirl {' '.join(args)}")
    completed = subprocess.run(cmd, capture_output=True, text=True)

    if completed.stdout:
        lines = completed.stdout.strip().splitlines()
        print("\n".join(lines[:max_lines]))
        if len(lines) > max_lines:
            print("...")

    if completed.stderr:
        err = completed.stderr.strip()
        if err:
            print("[stderr]", shorten(err, width=220, placeholder=" ..."))

    print(f"(exit={completed.returncode})\n")
    return completed.returncode

## 1. Python SDK (Sync) — `ProxyWhirl`

Use this for scripts and synchronous workloads.


In [ ]:
sync_proxies = [
    Proxy(
        url="http://proxy-us.example.com:8080",
        source=ProxySource.USER,
        country_code="US",
        health_status=HealthStatus.HEALTHY,
    ),
    Proxy(
        url="http://proxy-de.example.com:8080",
        source=ProxySource.USER,
        country_code="DE",
        health_status=HealthStatus.HEALTHY,
    ),
    Proxy(
        url="http://proxy-sg.example.com:8080",
        source=ProxySource.USER,
        country_code="SG",
        health_status=HealthStatus.HEALTHY,
    ),
]

sync_rotator = ProxyWhirl(proxies=[p.model_copy() for p in sync_proxies], strategy="round-robin")


def mock_sync_request(self, method, url, **kwargs):
    return httpx.Response(
        200,
        request=httpx.Request(method, url),
        json={"ok": True, "method": method, "url": str(url), "interface": "sync"},
    )


with patch.object(httpx.Client, "request", new=mock_sync_request):
    get_payload = sync_rotator.get("https://example.com/ip").json()
    post_payload = sync_rotator.post("https://example.com/data", json={"hello": "world"}).json()

print("GET:")
jprint(get_payload)
print("\nPOST:")
jprint(post_payload)
print("\nPer-proxy request totals:", [p.total_requests for p in sync_rotator.pool.get_all_proxies()])

## 2. Python SDK (Async) — `AsyncProxyWhirl`

Same API shape as sync (`get`, `post`, etc.), but async/await native.


In [ ]:
async def mock_async_request(self, method, url, **kwargs):
    return httpx.Response(
        200,
        request=httpx.Request(method, url),
        json={"ok": True, "method": method, "url": str(url), "interface": "async"},
    )


async def async_sdk_demo():
    rotator = AsyncProxyWhirl(
        proxies=[p.model_copy() for p in sync_proxies],
        strategy="random",
        retry_policy=RetryPolicy(max_attempts=2),
    )

    with patch.object(httpx.AsyncClient, "request", new=mock_async_request):
        async with rotator:
            payload = (await rotator.get("https://example.com/async")).json()
            stats = rotator.get_pool_stats()
    return payload, stats


async_payload, async_stats = run_async(async_sdk_demo())
print("Payload:")
jprint(async_payload)
print("\nStats keys:", sorted(async_stats.keys()))

## 3. Strategy Interface (Constructor + Hot Swap)

In [ ]:
strategy_names = [
    "round-robin",
    "random",
    "weighted",
    "least-used",
    "performance-based",
    "session",
    "geo-targeted",
]

print("Constructor aliases:")
for name in strategy_names:
    resolved = ProxyWhirl(strategy=name).strategy.__class__.__name__
    print(f"  {name:18} -> {resolved}")

swapped = ProxyWhirl(strategy="round-robin")
swapped.set_strategy("performance-based")
print("\nAfter set_strategy('performance-based'):", swapped.strategy.__class__.__name__)

## 4. Pool Maintenance + Statistics

In [ ]:
maintenance = ProxyWhirl(proxies=[p.model_copy() for p in sync_proxies])
maintenance.pool.proxies[0].health_status = HealthStatus.DEAD
maintenance.pool.proxies[1].health_status = HealthStatus.UNHEALTHY

removed = maintenance.clear_unhealthy_proxies()
print("Removed unhealthy:", removed)
print("Remaining size:", maintenance.pool.size)
print("\nPool stats:")
jprint(maintenance.get_pool_stats())
print("\nExtended stats:")
jprint(maintenance.get_statistics())

## 5. Fetch + Validate Pipeline

In [ ]:
from proxywhirl.fetchers import ProxyFetcher, ProxySourceConfig, ProxyValidator
from proxywhirl.models import ValidationLevel

sample_source_text = """8.8.8.8:8080
1.1.1.1:8080
8.8.8.8:8080
""".strip()


async def fetch_validate_demo():
    fetcher = ProxyFetcher(
        sources=[ProxySourceConfig(url="https://source.example/proxies.txt", format="plain_text")],
        validator=ProxyValidator(level=ValidationLevel.BASIC, concurrency=4),
    )

    async def fake_get(self, url, *args, **kwargs):
        return httpx.Response(
            200,
            request=httpx.Request("GET", str(url)),
            text=sample_source_text,
        )

    with patch.object(httpx.AsyncClient, "get", new=fake_get):
        return await fetcher.fetch_all(validate=False)


fetched = run_async(fetch_validate_demo())
print(f"Fetched proxies: {len(fetched)}")
for item in fetched:
    print("  -", item.get("url"))

## 6. CLI Interface

In [ ]:
run_cli(["--help"], max_lines=20)
run_cli(["--no-lock", "config", "show"], max_lines=16)
run_cli(["--no-lock", "stats"], max_lines=14)

## 7. REST API Interface (FastAPI)

Uses in-process `TestClient` + dependency override for deterministic demo behavior.


In [ ]:
try:
    from fastapi.testclient import TestClient
except ImportError as exc:  # pragma: no cover
    raise ImportError("Install FastAPI extras: `pip install -U proxywhirl[api] fastapi`") from exc

from proxywhirl.api import app
from proxywhirl.api.core import get_rotator

api_rotator = ProxyWhirl(proxies=[Proxy(url="http://proxy-api.example.com:8080")])
app.dependency_overrides[get_rotator] = lambda: api_rotator


async def mock_api_target_request(self, method, url, *args, **kwargs):
    return httpx.Response(
        200,
        request=httpx.Request(method, url),
        text='{"api": true, "target": "ok"}',
        headers={"content-type": "application/json"},
    )


with TestClient(app) as client:
    status_payload = client.get("/api/v1/status").json()
    list_payload = client.get("/api/v1/proxies").json()

    with patch.object(httpx.AsyncClient, "request", new=mock_api_target_request):
        request_payload = client.post(
            "/api/v1/request",
            json={"url": "https://example.com", "method": "GET", "timeout": 10},
        ).json()

app.dependency_overrides.clear()

print("Status summary:")
jprint(status_payload.get("data", {}))
print("\nFirst proxy:")
jprint(list_payload.get("data", {}).get("items", [])[0])
print("\nProxied request result:")
jprint(request_payload.get("data", {}))

## 8. MCP Wrapper Interface

In [ ]:
from proxywhirl.mcp.server import (
    cleanup_rotator,
    get_proxy_health,
    list_proxies,
    recommend_proxy,
    rotate_proxy,
    set_rotator,
)


async def mcp_wrapper_demo():
    rotator = AsyncProxyWhirl(
        proxies=[
            Proxy(url="http://proxy-mcp-us.example.com:8080", country_code="US"),
            Proxy(url="http://proxy-mcp-de.example.com:8080", country_code="DE"),
        ]
    )
    await set_rotator(rotator)

    listing = await list_proxies()
    rotated = await rotate_proxy()
    recommended = await recommend_proxy(region="US", performance="high")
    health = await get_proxy_health()

    await cleanup_rotator()
    return listing, rotated, recommended, health


mcp_listing, mcp_rotated, mcp_recommended, mcp_health = run_async(mcp_wrapper_demo())
print("List summary:")
jprint({"total": mcp_listing.get("total"), "healthy": mcp_listing.get("healthy")})
print("\nrotate_proxy:")
jprint(mcp_rotated)
print("\nrecommend_proxy:")
jprint(mcp_recommended)
print("\nget_proxy_health:")
print(mcp_health)

## 9. Launch Snippets

In [ ]:
print("CLI:")
print("  uv run python -m proxywhirl.cli --help")
print("\nREST API server:")
print("  uv run python -m uvicorn proxywhirl.api:app --host 0.0.0.0 --port 8000")
print("\nMCP server module:")
print("  uv run python -m proxywhirl.mcp.server")
print("\nTUI:")
print("  uv run python -m proxywhirl.cli tui")

## Final Checklist

- SDK sync and async flows validated
- Strategy aliases and hot-swap validated
- CLI commands validated
- REST API paths validated
- MCP wrapper methods validated

You can now adapt each section for production endpoints and real proxy sources.
